In [3]:
import pandas as pd
import altair as alt
import numpy as np
import datetime as dt
from vega_datasets import data



In [4]:
df = pd.read_csv('billboard_200_cleaned.csv')

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [28]:
df['Song/Album Title'] = df['Song/Album Title'].str.replace(r'\[\d+\]', '', regex=True)

In [29]:
bts = df[df['Artist'] == 'BTS']

In [8]:
# bts.to_csv('bts.csv', index=False)

## Graph_BTS

### Annotations:
- In 2009, only one K-pop album - BOA - made it onto the Billboard 200 chart. The album peaked at position 127 and remained on the chart for just one week. 
- A decade later, nine albums made it onto the Billboard 200, including three releases from BTS. Two of these albums - Love Yourself: Tear and Love Yourself: Answer reached the number one spot and stayed on the chart for over 15 weeks.

### !!! TODO: 
1. insert: https://youtube.com/clip/UgkxX9Ulb1Ai5WpAdtRaz6b9lqbfytEHaLwu?si=8b0bvQJwIqKKKPyr
2. make the dots as lines


In [30]:
bts['date'] = pd.to_datetime(df['date'])

/var/folders/yz/t21562ys7x37ts3_y63c_6fm0000gn/T/ipykernel_14891/3018936341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bts['date'] = pd.to_datetime(df['date'])


In [31]:
bts.head()

,date,Artist,Song/Album Title,Peak Position,Weeks on Chart,year,generation,debut_years
10,2015-12-19,BTS,"The Most Beautiful Moment In Life, Part 2",171,1,2015,3,2012 - 2017
11,2016-05-21,BTS,The Most Beautiful Moment In Life: Young Forever,107,1,2016,3,2012 - 2017
12,2016-10-29,BTS,Wings,26,2,2016,3,2012 - 2017
14,2017-03-04,BTS,You Never Walk Alone,61,2,2017,3,2012 - 2017
17,2017-10-07,BTS,Love Yourself: Her,7,7,2017,3,2012 - 2017


In [72]:
# Create a selection for the click interaction
brush = alt.selection_interval(encodings=['x'])

# Create the timeline chart
chart = alt.Chart(bts).mark_bar().encode(
    x=alt.X('date:T', title='Date', axis=alt.Axis(
        labelAngle=-45,  # Rotate labels for readability
    )),
    y=alt.Y('Weeks on Chart:Q', title='Weeks on Chart'),
    color=alt.Color('Weeks on Chart:Q', title='Weeks on Chart', scale=alt.Scale(scheme='accent')),
    tooltip=[
        alt.Tooltip('Song/Album Title:N', title='Song/Album Title'),
        alt.Tooltip('Peak Position:Q', title='Peak Position'),
        alt.Tooltip('Weeks on Chart:Q', title='Weeks on Chart'),
        alt.Tooltip('year:O', title='Year'),
    ]
).add_selection(
    brush
).properties(
    title='Timeline of Songs/Albums on Chart',
    width=600,
    height=400
)

/var/folders/yz/t21562ys7x37ts3_y63c_6fm0000gn/T/ipykernel_14891/2970721905.py:17: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


In [73]:
# Create the scatter plot

scatter = alt.Chart(bts).mark_circle(size=100).encode(
    y=alt.Y('Peak Position:Q', title='Peak Position (Weeks)', scale=alt.Scale(reverse=True)),
    x=alt.X('Song/Album Title:N', sort='-y', title='Song/Album',axis=alt.Axis(
        labelAngle=-45,     # Rotate labels for readability
    )),
    opacity=alt.condition(brush, alt.value(1.0), alt.value(0.2)),
    color=alt.Color('Peak Position:Q', title='Peak Position', scale=alt.Scale(scheme='accent')),
    tooltip=['Song/Album Title:N', 'Peak Position:Q']
).transform_window(
    rank='rank(Peak Position)',
    sort=[alt.SortField('Peak Position', order='ascending')]
).add_selection(
    brush
).properties(
    width=300,
    height=150,
    title='Peak Position of Songs/Albums on Chart'
)



# Combine the line chart and bar chart
final_chart = alt.vconcat(chart, scatter)


/var/folders/yz/t21562ys7x37ts3_y63c_6fm0000gn/T/ipykernel_14891/3801873971.py:14: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


In [74]:
final_chart

alt.VConcatChart(...)

In [ ]:
# Create generation based on artist name 
# df['generation'] = np.where(df['Artist'] == 'BoA', 1, df['generation'])
# df['generation'] = np.where(df['Artist'].isin(['Big Bang', '2NE1', "Girls' Generation-TTS", "Girls' Generation", 'Taeyang', 'G-Dragon', "Jackson Wang", 'Jonghyun']), 2, df['generation'])
# df['generation'] = np.where(df['Artist'].isin(['Exo', 'BTS', "Blackpink", "Exo-K", 'The Rose', 'Seventeen', 'Twice', 'Red Velvet', 'Jimin', 'Jungkook', 'NCT 127', 'NCT Dream', 'NCT', 'J-Hope', 'Loona', 'Nayeon', 'Agust D', 'Jihyo', 'DPR Ian', 'SuperM', 'Monsta X', 'RM', 'V']), 3, df['generation'])
# df['generation'] = np.where(df['Artist'].isin(['Stray Kids', '(G)I-dle', 'TXT', 'Itzy', 'Ateez', 'Aespa', 'NewJeans', 'Enhypen', 'Le Sserafim', 'Nmixx', 'P1Harmony', 'Kep1er' ]), 4, df['generation'])
# df['generation'] = np.where(df['Artist'].isin(['Babymonster', 'Illit', 'BoyNextDoor', 'Xikers']), 5, df['generation'])
  